In [1]:
# The Oracle 
# V2.6 2024-10-10

import re
import os
import base64
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from utils.common_utils import print_help
from ioc_processing.ioc_functions import (
    #validate_iocs,
    parse_bulk_iocs,
    process_individual_ioc_file,
    analysis,
    calculate_total_malicious_score
)
from file_operations.file_utils import read_file, clean_input, is_ip, is_url, is_domain, is_hash, sanitize_and_defang


# Function to list files in the input_files directory
def list_input_files():
    input_dir = 'input_files'
    return [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# Check for light or dark mode and update text color accordingly
js_code = """
<script>
    function applyColorScheme() {
        const isDarkMode = window.matchMedia('(prefers-color-scheme: dark)').matches;
        const elements = document.querySelectorAll('body, .custom-box, .widget-radio-box, .widget-button, .widget-checkbox label, .widget-textarea');

        elements.forEach(el => {
            if (isDarkMode) {
                el.classList.remove('light-mode');
                el.classList.add('dark-mode');
            } else {
                el.classList.remove('dark-mode');
                el.classList.add('light-mode');
            }
        });
    }

    window.matchMedia('(prefers-color-scheme: dark)').addEventListener('change', applyColorScheme);
    applyColorScheme();  // Initial call to set the colors based on current theme
</script>
"""
display(HTML(js_code))


# Function to classify the IOC based on the detected pattern
def classify_ioc(ioc):
    ioc = ioc.strip()  # Clean up whitespace
    if is_ip(ioc):
        return 'ip'
    elif is_url(ioc):
        return 'url'
    elif is_domain(ioc):
        return 'domain'
    elif is_hash(ioc):
        return 'hash'
    else:
        return 'unknown'

# Function to handle validation and display results
def start_validation(output_to_file, custom_output_filename=None, raw_input=None, file_input=None, file_name_input=None, progress_bar=None, status_output=None):
    iocs = []

    # Handling raw input (comma-separated or single)
    if raw_input:
        print(f"DEBUG: Raw input before cleaning:\n{raw_input}")
        # Clean the input if necessary
        cleaned_input = sanitize_and_defang(raw_input, defang=False)
        print(f"DEBUG: Cleaned input:\n{cleaned_input}")
        # Split the input on commas
        iocs = [ioc.strip() for ioc in cleaned_input.split(',') if ioc.strip()]
        print(f"DEBUG: IOCs after splitting on commas:\n{iocs}")

    # Handling file input (uploaded file or dropdown selected file content)
    if file_input:
        print(f"DEBUG: Uploaded/Selected file content:\n{file_input}")
        iocs = [ioc.strip() for ioc in file_input.splitlines() if ioc.strip()]  # Split into individual IOCs

    # Proceed only if IOCs are found
    if not iocs:
        with status_output:
            clear_output()
            display(HTML('<b>Error: No valid IOCs found.</b>'))
        return

    # Automatically classify IOCs
    ioc_dict = {'ips': [], 'urls': [], 'domains': [], 'hashes': []}
    for ioc in iocs:
        ioc_type = classify_ioc(ioc)
        if ioc_type != 'unknown':
            # Handle the plural case for "hash"
            if ioc_type == "hash":
                ioc_dict['hashes'].append(ioc)
            else:
                ioc_dict[f'{ioc_type}s'].append(ioc)

    total_api_calls = len(ioc_dict['ips']) * 10 + len(ioc_dict['urls']) * 9 + len(ioc_dict['domains']) * 9 + len(ioc_dict['hashes']) * 6

    # Filter out empty lists
    ioc_dict = {k: v for k, v in ioc_dict.items() if v}

    if not ioc_dict:
        with status_output:
            clear_output()
            display(HTML('<b>Error: No valid IOCs found after classification.</b>'))
        return

    print(f"DEBUG: Starting analysis with IOCs = {ioc_dict}")

    # Display progress bar before starting analysis
    if progress_bar and status_output:
        with status_output:
            clear_output()
            display(HTML('<b>Performing analysis...</b>'))
            progress_container = widgets.HBox(
                [progress_bar],
                layout=widgets.Layout(justify_content='flex-start', width='100%')
            )
            display(progress_container)

    # Proceed with analysis
    aggregated_report = analysis(
        selected_category=ioc_dict,
        output_file_path=custom_output_filename,
        progress_bar=progress_bar,
        status_output=status_output
    )

    # Save output to file if specified
    if output_to_file and custom_output_filename:
        with open(custom_output_filename, "a") as outfile:
            outfile.write(aggregated_report)
            outfile.write("\n")

    # Display the aggregated report
    with output:
        clear_output()
        print(aggregated_report)

    # Indicate completion
    if status_output:
        with status_output:
            clear_output()
            display(progress_bar)
            display(HTML('<b>Processing complete.</b>'))

# Function to reset the entire form including file inputs
def full_reset_inputs(b):
    output_to_file_checkbox.value = False
    file_input.value = ()  # Reset file input
    file_name_input.value = None  # Reset dropdown selection
    custom_file_name.value = ''  # Reset custom file name
    raw_input.value = ''
    with output:
        clear_output()
    with status_output:
        clear_output()
    processing_label.value = ""  # Clear the processing message
    print("Full reset completed.")  # Debugging message

full_reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='danger',
    tooltip='Click to fully reset inputs',
    icon='trash',
    layout=widgets.Layout(width='500px', height='auto')
)

# Create widgets with custom styles
style = {'description_width': 'initial'}
layout = widgets.Layout(width='auto')

output_to_file_checkbox = widgets.Checkbox(
    value=False,
    description='Save output to file',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='250px', height='auto', padding='18px', overflow='hidden', background_color='transparent', border_radius='10px')  # Adjust height and overflow
)

custom_file_name = widgets.Text(
    value='',
    placeholder='File Name (no ext)',
    description='Output File Name:',
    style=style,
    layout=widgets.Layout(width='270px', background_color='transparent')
)

# Adjust the layout of the VBox containing the checkbox
checkbox_vbox = widgets.VBox(
    [output_to_file_checkbox, custom_file_name],  # Added custom file name widget
    layout=widgets.Layout(padding='5px', background_color='#333333', border_radius='10px', overflow='hidden')  # Ensure no scroll bars by setting overflow to hidden
)

file_input = widgets.FileUpload(
    accept='.txt',  # Accept only .txt files
    multiple=False,  # Accept single file
    style=style,
    layout=widgets.Layout(width='275px')
)

file_name_input = widgets.Dropdown(
    options=list_input_files(),
    value=None,
    placeholder='From input_files directory',
    description='File Name:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='275px', background_color='transparent')
)

raw_input = widgets.Text(
    value='',
    placeholder='(ie. IP, URL, Hash)',
    description='Raw Input:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='275px', background_color='transparent')
)

submit_button = widgets.Button(
    description='Start Validation',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to start validation',
    icon='check',  # (FontAwesome names without the `fa-` prefix)
    style=style,
    layout=widgets.Layout(width='275px')
)

# Load image
image_path = 'oracle_logo.jpg'  # Replace with your actual image path
with open(image_path, "rb") as file:
    image_data = file.read()
    encoded_image = base64.b64encode(image_data).decode('utf-8')

# Create an image widget
image_widget = widgets.Image(
    value=image_data,
    format='jpg',
    width=400,
    height=400,
    layout=widgets.Layout(margin='160px 0 0 0')
)

# Load the banner image
banner_img_path = "The_Oracle_Banner_IOC_Validation_Tool2.jpg"
with open(banner_img_path, "rb") as file:
    banner_image_data = file.read()

# Create an image widget for the banner
banner_widget = widgets.Image(
    value=banner_image_data,
    format='jpeg',
    width=800,
    layout=widgets.Layout(margin='0 0 0px 0')
)

# Initialize the progress bar widget
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', layout=widgets.Layout(width='90%', align_self='flex-start'))


processing_label = widgets.Label(value="", layout=widgets.Layout(width='250px', margin='-5px 0 0 0'))  # Label to indicate processing status

output = widgets.Output(layout=widgets.Layout(width='100%', height='1100px', overflow='auto'))  # Increased height to 900px
status_output = widgets.Output(layout=widgets.Layout(width='100%', height='100px'))  # Adjusted height to fit status and progress bar

# Define the interaction logic
def on_submit_button_clicked(b):
    output_to_file = output_to_file_checkbox.value
    raw_text = raw_input.value.strip()  # Get raw text input and trim whitespace

    # Initialize output_file_path
    output_file_path = None

    # Determine if output should be saved to a file
    if output_to_file:
        if custom_file_name.value:
            output_file_path = f"output_files/{custom_file_name.value}.txt"
        else:
            output_file_path = f"output_files/default_output.txt"

    # Initialize variables for file content
    uploaded_file_content = None
    dropdown_file_content = None

    # Handling file upload (from the FileUpload widget)
    if file_input.value:
        print("DEBUG: file_input.value =", file_input.value)  # Add detailed debugging to see the structure
        try:
            # Extract the uploaded file content (since file_input.value is a tuple containing the file dictionary)
            uploaded_file = file_input.value[0]  # Get the first element of the tuple
            uploaded_file_content = uploaded_file['content'].tobytes().decode('utf-8')  # Convert memoryview to bytes and then decode to string
        except Exception as e:
            with status_output:
                clear_output()
                display(HTML(f"<b>Error extracting uploaded file content: {e}</b>"))
            return

    # Handling dropdown file selection (from the dropdown menu)
    if file_name_input.value and not file_input.value:
        # Only process dropdown selection if no file has been uploaded
        file_name_text = file_name_input.value
        file_path = os.path.join('input_files', file_name_text)

        # Ensure the file exists before attempting to read it
        if os.path.isfile(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    dropdown_file_content = f.read()
            except Exception as e:
                with status_output:
                    clear_output()
                    display(HTML(f"<b>Error reading file from dropdown: {e}</b>"))
                return
        else:
            with status_output:
                clear_output()
                display(HTML(f"<b>Error: File {file_name_text} not found.</b>"))
            return

    # Refang the raw input (defanged IP, URL, etc.)
    refanged_raw_text = sanitize_and_defang(raw_text, defang=False)

    # Ensure there is at least one valid input provided
    if not refanged_raw_text and not uploaded_file_content and not dropdown_file_content:
        with status_output:
            clear_output()
            display(HTML("<b>Error: Please provide an IOC input or select a file.</b>"))
        return

    # Choose the appropriate file content if provided
    file_content = uploaded_file_content if uploaded_file_content else dropdown_file_content

    # Pass the output_file_path to start_validation
    start_validation(
        output_to_file=output_to_file,
        raw_input=refanged_raw_text,
        file_input=file_content,  # Pass the processed file content (either from upload or dropdown)
        file_name_input=None,  # We don't need to pass the file name since content is already handled
        progress_bar=progress_bar,
        status_output=status_output
    )

# Bind the button click events to the functions
submit_button.on_click(on_submit_button_clicked)
#partial_reset_button.on_click(partial_reset_inputs)
full_reset_button.on_click(full_reset_inputs)

# Create an HTML widget for the classification text with explicit centering
classification_text = widgets.HTML(
    value="<h3 style='text-align: center;'>UNCLASSIFIED//OUO</h3>",
    layout=widgets.Layout(margin='0px 0px 10px 0px', width='100%')
)


# Arrange the UI components in a box with a grey background
input_widgets = widgets.VBox([
    banner_widget,
    classification_text,
    raw_input,
    file_name_input,
    file_input,
    checkbox_vbox,
    submit_button,
    widgets.HBox([full_reset_button]),  # Both reset buttons in a horizontal box
    processing_label,
    status_output,
    image_widget,
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', background_color='#333333', border_radius='10px'))

app_layout = widgets.AppLayout(
    left_sidebar=input_widgets,
    center=None,
    right_sidebar=output,
    pane_widths=['300px', 0, 1],  # Reduced width of the left sidebar
    pane_heights=['80px', '1100px', '60px']  # Increased height to 900px
)

display(app_layout)

AppLayout(children=(VBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x0…